In [14]:
import numpy as np
import pandas as pd

# Load ratings Change directory where appropriate
directory = 'ml-latest-small/'
ratings = pd.read_csv(directory+'ratings.csv', usecols=['userId', 'movieId', 'rating'])
movies = pd.read_csv(directory+'movies.csv', usecols=['movieId', 'title', 'genres'])
max_userid = ratings['userId'].drop_duplicates().max()+1
max_movieid = ratings['movieId'].drop_duplicates().max() +1
ratings.head()
print(max_userid)
print(max_movieid)

611
193610


In [15]:
len(ratings_list)

100836

In [16]:

from keras.layers import Embedding, Reshape, Concatenate,dot,Input,Dense
from keras.models import Model,clone_model
from keras.callbacks import Callback, EarlyStopping, ModelCheckpoint


factors = 100 

user_ids = ratings['userId'].values
movie_ids = ratings['movieId'].values
ratings_list = ratings['rating'].values



In [17]:
def model_maker(max_userid = max_userid,max_movieid = max_movieid,factors=factors):
# user_layers is the embedding layer that creates an User by latent factors matrix.

    user_input = Input(shape=(1,), name = 'user_layers')
    user_middle  = Embedding(max_userid, factors,input_length=1)(user_input)
    user_layers  = Reshape(target_shape=(factors,))(user_middle)


    # movie_layers is the embedding layer that creates a Movie by latent factors matrix.
    movie_input = Input(shape=(1,), name = 'movie_layers')
    movie_middle  = Embedding(max_movieid, factors,input_length=1)(movie_input)
    movie_layers  = Reshape(target_shape=(factors,))(movie_middle)

    output = dot([user_layers,movie_layers], axes=1)

    model = Model(inputs=[user_input,movie_input], 
                      outputs=output)
    model.compile(loss='mse', optimizer='adamax')

    return model

In [28]:
from random import choice,sample
rando = sample(range(0,len(user_ids)-1),1000)
#print(rando)
short_user=user_ids[rando]
short_movie=movie_ids[rando]
short_rate = ratings_list[rando]


In [13]:
fp = open("output.txt",'w')
#print(type(fp))
fp.write('test'+'\n')
for i in short_rate:
    fp.write(str(i)+'\n')
fp.close()

In [29]:
from sklearn.model_selection import KFold
import csv
import math

#noramlly random, but its for reprodcbility
kfold = KFold(n_splits=7, shuffle=True, random_state=np.random.seed(10))
scores = []
rmses = []
min_epochs = []
i = 0
for train, test in kfold.split(ratings_list):
    
    i+=1 
    print("Fold {}".format(i))
    weight_string = 'test/weights'+str(i)+'.h5'
    model = model_maker()
    callbacks = [EarlyStopping('val_loss', patience=2),  ModelCheckpoint(weight_string, save_best_only=True)]
    history = model.fit([user_ids[train],movie_ids[train]],ratings_list[train], epochs=25, validation_split=.1, verbose=2, callbacks=callbacks)
      
    min_val_loss, idx = min((val, idx) for (idx, val) in enumerate(history.history['val_loss']))
    rmses.append(min_val_loss)
    min_epochs.append(idx)
    print( 'Minimum RMSE at epoch ' + '{:d}'.format(idx+1) + ' = '+ '{:.4f}'.format(math.sqrt(min_val_loss)))
    score = model.evaluate([user_ids[test],movie_ids[test]],ratings_list[tst])
    print('Loss is: {}'.format(score))
    scores.append(score)



Fold 1
Train on 771 samples, validate on 86 samples
Epoch 1/25
 - 15s - loss: 13.5145 - val_loss: 13.0657
Epoch 2/25
 - 11s - loss: 13.4360 - val_loss: 13.0663
Epoch 3/25
 - 11s - loss: 13.3747 - val_loss: 13.0671
Minimum RMSE at epoch 1 = 3.6146
143/143 [==============================] - 0s 223us/step
Loss is: 13.909259702775861
Fold 2
Train on 771 samples, validate on 86 samples
Epoch 1/25
 - 14s - loss: 13.5421 - val_loss: 12.8768
Epoch 2/25
 - 12s - loss: 13.4646 - val_loss: 12.8773
Epoch 3/25
 - 13s - loss: 13.4031 - val_loss: 12.8775
Minimum RMSE at epoch 1 = 3.5884
143/143 [==============================] - 0s 251us/step
Loss is: 13.866465081701746
Fold 3
Train on 771 samples, validate on 86 samples
Epoch 1/25
 - 15s - loss: 13.7312 - val_loss: 13.0828
Epoch 2/25
 - 12s - loss: 13.6566 - val_loss: 13.0824
Epoch 3/25
 - 12s - loss: 13.5956 - val_loss: 13.0824
Epoch 4/25
 - 11s - loss: 13.5338 - val_loss: 13.0824
Epoch 5/25
 - 12s - loss: 13.4685 - val_loss: 13.0828
Minimum RMSE a

In [8]:
rmses = [math.sqrt(item) for item in rmses]
print("{:.4f}% (+/- {:.4f}%)".format(np.mean(scores), np.std(scores)))
print(scores)

print("{:.4f}% (+/- {:.4f}%)".format(np.mean(rmses), np.std(rmses)))
print(rmses)

fp = open("output.csv",'w')
#print(type(fp))
fp.write('fold,min_epoch,rmse,loss_score'+'\n')
for i in range(len(scores)):
    fp.write(str(i)+','+str(min_epochs[i])+','+str(rmses[i])+','+str(scores[i])+','+'\n')
fp.close()

1.5415% (+/- 0.0120%)
[1.5378864061997248, 1.5474179587334433, 1.561328424377157, 1.5430102679342337, 1.519050771275815, 1.5465742985417397, 1.5349368303138387]
1.8340% (+/- 0.0030%)
[1.836997025225401, 1.8329663511431422, 1.835992808685128, 1.837312900144314, 1.8281652086119626, 1.8345682820374172, 1.8320836752015814]


In [ ]:
# Use the pre-trained model
trained_model = clone_model(model)
# Load weights
trained_model.load_weights('movie_weights.h5')

In [ ]:
def predict_rating(user_id, movie_id):
    return trained_model.predict([np.array([user_id]), np.array([movie_id])])[0][0]


In [ ]:

rand_user = 450#choice(user_ids)
rand_movie = choice(movie_ids)

user_ratings = ratings[ratings['userId'] == rand_user][['userId', 'movieId', 'rating']]
#print(ratings[ratings['userId'] == rand_user][['userId', 'movieId','rating']])
user_ratings['prediction'] = user_ratings.apply(lambda x: predict_rating(rand_user, x['movieId']), axis=1)
user_ratings.sort_values(by='rating', 
                         ascending=False).merge(movies, 
                                                on='movieId', 
                                                how='inner', 
                                                suffixes=['_u', '_m']).head(20)

#print(user_ratings['prediction'].values.shape)


In [ ]:
recommendations = ratings[ratings['movieId'].isin(user_ratings['movieId']) == False][['movieId']].drop_duplicates()
recommendations['prediction'] = recommendations.apply(lambda x: predict_rating(rand_user, x['movieId']), axis=1)
recommendations.sort_values(by='prediction',
                          ascending=False).merge(movies,
                                                 on='movieId',
                                                 how='inner',
                                                 suffixes=['_u', '_m']).head(20)

In [ ]:
matrix = np.zeros((movie_ids.size,1))
#print(matrix)
for user in user_ids:
    newcol = np.array([predict_rating(user, movie) for movie in movie_ids])
    newcol.shape=((movie_ids.size,1))
    matrix = np.hstack((matrix,newcol))
matrix = matrix[:,1:]
print(matrix)
